In [1]:
!apt-get install openjdk-17-jdk-headless -qq > /dev/null


In [2]:
!java -version


openjdk version "17.0.16" 2025-07-15
OpenJDK Runtime Environment (build 17.0.16+8-Ubuntu-0ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 17.0.16+8-Ubuntu-0ubuntu122.04.1, mixed mode, sharing)


In [3]:
%%writefile Main.java
public class Main {
    public static void main(String[] args) {
        System.out.println("Java is working in Colab!");
    }
}


Writing Main.java


In [4]:
!javac Main.java


In [5]:
!java Main


Java is working in Colab!


In [6]:
%%writefile Vehicle.java
public abstract class Vehicle {
    protected String vehicleId;
    protected String brand;
    protected String model;
    protected double rentalPricePerDay;
    protected boolean isRented;

    public Vehicle(String vehicleId, String brand, String model, double rentalPricePerDay) {
        this.vehicleId = vehicleId;
        this.brand = brand;
        this.model = model;
        this.rentalPricePerDay = rentalPricePerDay;
        this.isRented = false;
    }

    public boolean isRented() {
        return isRented;
    }

    public void rent() {
        isRented = true;
    }

    public void returnVehicle() {
        isRented = false;
    }

    public String getVehicleId() {
        return vehicleId;
    }

    public double getRentalPricePerDay() {
        return rentalPricePerDay;
    }

    public abstract void displayInfo();
}


Writing Vehicle.java


In [7]:
%%writefile Car.java
public class Car extends Vehicle {
    private int numberOfSeats;
    private String fuelType;

    public Car(String vehicleId, String brand, String model, double rentalPricePerDay, int seats, String fuelType) {
        super(vehicleId, brand, model, rentalPricePerDay);
        this.numberOfSeats = seats;
        this.fuelType = fuelType;
    }

    @Override
    public void displayInfo() {
        System.out.println("Car - ID: " + vehicleId + ", " + brand + " " + model +
                ", Seats: " + numberOfSeats + ", Fuel: " + fuelType +
                ", Price/Day: " + rentalPricePerDay +
                ", Rented: " + (isRented ? "Yes" : "No"));
    }
}


Writing Car.java


In [8]:
%%writefile Bike.java
public class Bike extends Vehicle {
    private int engineCC;

    public Bike(String vehicleId, String brand, String model, double rentalPricePerDay, int engineCC) {
        super(vehicleId, brand, model, rentalPricePerDay);
        this.engineCC = engineCC;
    }

    @Override
    public void displayInfo() {
        System.out.println("Bike - ID: " + vehicleId + ", " + brand + " " + model +
                ", Engine: " + engineCC + "cc" +
                ", Price/Day: " + rentalPricePerDay +
                ", Rented: " + (isRented ? "Yes" : "No"));
    }
}


Writing Bike.java


In [9]:
%%writefile Customer.java
import java.util.ArrayList;
import java.util.List;

public class Customer {
    private String customerId;
    private String name;
    private List<Vehicle> rentedVehicles;

    public Customer(String customerId, String name) {
        this.customerId = customerId;
        this.name = name;
        this.rentedVehicles = new ArrayList<>();
    }

    public String getName() {
        return name;
    }

    public void rentVehicle(Vehicle v) {
        rentedVehicles.add(v);
    }

    public void returnVehicle(Vehicle v) {
        rentedVehicles.remove(v);
    }

    public boolean hasRented(Vehicle v) {
        return rentedVehicles.contains(v);
    }
}


Writing Customer.java


In [10]:
%%writefile RentalService.java
import java.util.ArrayList;
import java.util.List;

public class RentalService {
    private List<Vehicle> vehicles = new ArrayList<>();

    public void addVehicle(Vehicle v) {
        vehicles.add(v);
    }

    public void showAvailableVehicles() {
        for (Vehicle v : vehicles) {
            if (!v.isRented()) {
                v.displayInfo();
            }
        }
    }

    public Vehicle findVehicle(String vehicleId) {
        for (Vehicle v : vehicles) {
            if (v.getVehicleId().equals(vehicleId)) {
                return v;
            }
        }
        return null;
    }

    public boolean rentVehicle(Customer c, String vehicleId, int days) {
        Vehicle v = findVehicle(vehicleId);
        if (v == null || v.isRented()) {
            return false;
        }

        v.rent();
        c.rentVehicle(v);

        System.out.println("Total Cost: ₹" + (v.getRentalPricePerDay() * days));
        return true;
    }

    public boolean returnVehicle(Customer c, String vehicleId) {
        Vehicle v = findVehicle(vehicleId);
        if (v == null || !c.hasRented(v)) {
            return false;
        }

        v.returnVehicle();
        c.returnVehicle(v);
        return true;
    }
}


Writing RentalService.java


In [11]:
%%writefile Main.java
import java.util.Scanner;

public class Main {
    public static void main(String[] args) {
        Scanner sc = new Scanner(System.in);

        RentalService service = new RentalService();
        Customer customer = new Customer("C001", "Saketh");

        service.addVehicle(new Car("CAR1", "Toyota", "Innova", 2000, 7, "Diesel"));
        service.addVehicle(new Car("CAR2", "Maruti", "Swift", 1200, 5, "Petrol"));
        service.addVehicle(new Bike("BIKE1", "Yamaha", "R15", 800, 155));
        service.addVehicle(new Bike("BIKE2", "Royal Enfield", "Classic", 900, 350));

        while (true) {
            System.out.println("\n===== VEHICLE RENTAL SYSTEM =====");
            System.out.println("1. Show available vehicles");
            System.out.println("2. Rent a vehicle");
            System.out.println("3. Return a vehicle");
            System.out.println("4. Exit");
            System.out.print("Enter choice: ");
            int choice = sc.nextInt();

            if (choice == 1) {
                service.showAvailableVehicles();
            }
            else if (choice == 2) {
                System.out.print("Enter Vehicle ID: ");
                String id = sc.next();
                System.out.print("Enter number of days: ");
                int days = sc.nextInt();

                if (service.rentVehicle(customer, id, days)) {
                    System.out.println("Rented Successfully!");
                } else {
                    System.out.println("Vehicle not available.");
                }
            }
            else if (choice == 3) {
                System.out.print("Enter Vehicle ID: ");
                String id = sc.next();

                if (service.returnVehicle(customer, id)) {
                    System.out.println("Returned Successfully!");
                } else {
                    System.out.println("Invalid Return.");
                }
            }
            else if (choice == 4) {
                System.out.println("Thank you for using the system!");
                break;
            }
            else {
                System.out.println("Invalid choice!");
            }
        }

        sc.close();
    }
}


Overwriting Main.java


In [12]:
!javac *.java


In [14]:
!java Main



===== VEHICLE RENTAL SYSTEM =====
1. Show available vehicles
2. Rent a vehicle
3. Return a vehicle
4. Exit
Enter choice: 3
Enter Vehicle ID: car1
Invalid Return.

===== VEHICLE RENTAL SYSTEM =====
1. Show available vehicles
2. Rent a vehicle
3. Return a vehicle
4. Exit
Enter choice: 2
Enter Vehicle ID: car1
Enter number of days: 6
Vehicle not available.

===== VEHICLE RENTAL SYSTEM =====
1. Show available vehicles
2. Rent a vehicle
3. Return a vehicle
4. Exit
Enter choice: 3
Enter Vehicle ID: car1
Invalid Return.

===== VEHICLE RENTAL SYSTEM =====
1. Show available vehicles
2. Rent a vehicle
3. Return a vehicle
4. Exit
Enter choice: 3
Enter Vehicle ID: CAR1
Invalid Return.

===== VEHICLE RENTAL SYSTEM =====
1. Show available vehicles
2. Rent a vehicle
3. Return a vehicle
4. Exit
Enter choice: 2
Enter Vehicle ID: CAR1
Enter number of days: 1
Total Cost: ₹2000.0
Rented Successfully!

===== VEHICLE RENTAL SYSTEM =====
1. Show available vehicles
2. Rent a vehicle
3. Return a vehicle
4. Ex